In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
from dash.dependencies import Output, Input

C:\Users\fenix\AppData\Local\Temp/ipykernel_1348/1750965824.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\fenix\AppData\Local\Temp/ipykernel_1348/1750965824.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
data = pd.read_csv('dash_noyear.csv')
#data["Date"]= pd.to_datetime(df["Date"], format = '%Y%m%d')
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   region                   171 non-null    object        
 1   Date                     171 non-null    datetime64[ns]
 2   Real_Values              171 non-null    float64       
 3   Predicted_Values_Linear  171 non-null    float64       
 4   Predicted_Values_RF      171 non-null    float64       
 5   Predicted_Values_DT      171 non-null    float64       
 6   Predicted_Values_KNN     171 non-null    float64       
 7   Predicted_Values_ANN     171 non-null    float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 10.8+ KB


In [3]:
#import dash
#import dash_core_components as dcc
#import dash_html_components as html
#import pandas as pd
#import numpy as np
#from dash.dependencies import Output, Input

#data = pd.read_csv('dash_.csv')

external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
        "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "NPLs in the EU"

app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.P(children="🏦", className="header-emoji",style={'textAlign':'center',"fontSize": "48px"}),
                html.H1(
                    children="NPLs Predictions", className="header-title",style={'textAlign':'center',"fontSize": "48px", "color": "black"}
                ),
                html.P(
                    children="Analyze the behavior of NPLs in the EU"
            #" and the number of solids content in the EU"
            " between 2014 and 2022",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Country", className="menu-title"),
                        dcc.Dropdown(
                            id="region-filter",
                            options=[
                                {"label": region, "value": region}
                                for region in np.sort(data.region.unique())
                            ],
                            value="Ireland",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
               # html.Div(
                    #children=[
                        #html.Div(children="Type", className="menu-title"),
                        #dcc.Dropdown(
                            #id="type-filter",
                            #options=[
                                #{"label": Dairy_product, "value": Dairy_product}
                                #for Dairy_product in data.type.unique()
                            #],
                            #value="Raw cows milk delivered to dairies",
                            #clearable=False,
                            #searchable=False,
                            #className="dropdown",
                        #),
                    #],
                #),
                html.Div(
                    children=[
                        html.Div(
                            children="Date Range",
                            className="menu-title"
                            ),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=data.Date.min().date(),
                            max_date_allowed=data.Date.max().date(),
                            start_date=data.Date.min().date(),
                            end_date=data.Date.max().date(),
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="Real-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="ANN-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),html.Div(
                    children=dcc.Graph(
                        id="RF-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="KNN-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="Linear-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),html.Div(
                    children=dcc.Graph(
                        id="DT-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),
    ]
)


@app.callback(
    [Output("Real-chart", "figure"),Output("ANN-chart", "figure"),Output("RF-chart", "figure"),Output("KNN-chart", "figure"),\
     Output("Linear-chart", "figure"),Output("DT-chart", "figure")],
    [
        Input("region-filter", "value"),
        #Input("type-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ],
)
def update_charts(region, start_date, end_date):
    mask = (
        (data.region == region)
        #& (data.type == milk_type)
        & (data.Date >= start_date)
        & (data.Date <= end_date)
    )
    filtered_data = data.loc[mask, :]
    Real_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["Real_Values"],
                "type": "lines",
                #"hovertemplate": "$%{y:.2f}<extra></extra>",
            },
        ],
        "layout": {
            "title": {
                "text": "NPL",
                "x": 0.05,
                "xanchor": "left",
            },
            "xaxis": {"fixedrange": True},
            #"yaxis": {"tickprefix": "$", "fixedrange": True},
            "colorway": ["#00BFFF"],#17B897
        },
    }
    
    ANN_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["Predicted_Values_ANN"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Predicted_Values_ANN", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#00BFFF"],##00BFFF
        },
    }
        
    RF_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["Predicted_Values_RF"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Predicted_Values_RF", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#2d39e1"],##00BFFF
        },
    }
    
    KNN_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["Predicted_Values_KNN"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Predicted_Values_KNN", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#E12D39"],
        },
    }

    Linear_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["Predicted_Values_Linear"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Predicted_Values_Linear", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#2de1d5"],
        },
    }
    
    DT_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["Predicted_Values_DT"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Predicted_Values_DT", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#17B897"],##c90076 #17B897 #f1c232
        },
    }    
    
    return Real_chart_figure, ANN_chart_figure, RF_chart_figure, KNN_chart_figure, Linear_chart_figure, DT_chart_figure


if __name__ == '__main__':
    app.run_server(host='localhost',port=8005)

Dash is running on http://localhost:8005/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8005/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2023 22:52:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 22:52:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 22:52:37] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 22:52:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 22:52:37] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 22:52:37] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 22:52:37] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 22:52:37] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -
